In [9]:
# Graphical Abstract Generator for Dynan Experiments (NBISC)
# Author: Hari Parthasarathy
# Description: Parses radiation biology experiment metadata from Excel and outputs a publication-ready Gantt chart

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [11]:
# === STEP 1: Load Excel Spreadsheet ===
file_path = "Comparisons (Mar 3rd).xlsx"
xls = pd.ExcelFile(file_path)

# Load the relevant sheet ("Dynan Si-28")
df = pd.read_excel(xls, sheet_name="Dynan Si-28")

# Rename columns for consistency
df.rename(columns={
    "Termination Date Final": "Termination Date",
    "Termination Date Initial": "Termination Date Initial"
}, inplace=True)

In [12]:
# === STEP 2: Parse and Clean Data ===
date_cols = ["Irradiation Date", "Termination Date", "Termination Date Initial", "DOB"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

df["Order"] = pd.to_numeric(df["Order"], errors='coerce')

# Sort by defined Order, then alphabetically by Group
df.sort_values(by=["Order", "Group"], inplace=True)


# === STEP 2.1: Parse and Clean Data (IF USING SLIMS OUTPUT) ===

# Load and preprocess the data
# df = pd.read_excel(xls, sheet_name="Dynan GCRSIM (SLIMS)")

# df["Time Point of Sacrifice Post Irradiation"] = pd.to_timedelta(df["Time Point of Sacrifice Post Irradiation"], unit='D')
# df["Irradiation Date"] = df["Date of Euthanasia"] - df["Time Point of Sacrifice Post Irradiation"]
# df["Irradiation Date"] = pd.to_datetime(df["Irradiation Date"], errors='coerce')
# df["Termination Date"] = pd.to_datetime(df["Date of Euthanasia"], errors='coerce')

# df["DOB"] = df["Irradiation Date"] - pd.to_timedelta(df["Age at Irradiation"], unit='W')
# df["DOB"] = pd.to_datetime(df["DOB"], errors='coerce')
# df["Order"] = pd.to_numeric(df["Order"], errors='coerce')

# df["Recovery Days"] = (df["Termination Date"] - df["Irradiation Date"]).dt.days

In [13]:
# === STEP 3: Create Composite Category for Legend ===
treatment_positive_term = "NR+"
df["Legend_Category"] = df.apply(
    lambda row: f"{row['Gender']} | {row['Beam']} | {'NR+' if row['Treatment'] == treatment_positive_term else 'NR-'} | {row['Group']}",
    axis=1
)

In [14]:
# === STEP 4: Define Color & Opacity Mapping ===
base_colors = {"Male": "#4C72B0", "Female": "#E9967A"}

def assign_color(row):
    return base_colors.get(row["Gender"], "#999999")

def assign_treatment_color(row):
    return "#0D98BA" if row["Treatment"] == treatment_positive_term else "#F7BAA8"

df["Color"] = df.apply(assign_color, axis=1)
df["Treatment_Color"] = df.apply(assign_treatment_color, axis=1)
df["Opacity"] = df["Beam"].apply(lambda x: 0.2 if "Control" in str(x) else 1.0)

# Store initial group order
default_y_order = df["Group"].unique().tolist()


In [15]:
# === STEP 5: Create Primary Gantt Chart ===
fig = px.timeline(
    df,
    x_start="Irradiation Date",
    x_end="Termination Date",
    y="Group",
    color="Legend_Category",
    color_discrete_map={cat: df[df["Legend_Category"] == cat]["Color"].values[0] for cat in df["Legend_Category"].unique()},
    text=df.apply(lambda row: f"{row['Subjects']} | {row['Dose']} cGy", axis=1),
    hover_data=["Beam", "Dose", "Treatment", "DOB"],
    title="Graphical Abstract for Dynan Experiments"
)

# === STEP 6: Add DOB → Irradiation Gray Bars ===
dob_df = df.copy()
dob_df["Color"] = "rgba(128, 128, 128, 0.2)"
dob_traces = px.timeline(
    dob_df,
    x_start="DOB",
    x_end="Irradiation Date",
    y="Group",
    color="Legend_Category",
    color_discrete_map={cat: "rgba(128, 128, 128, 0.2)" for cat in dob_df["Legend_Category"].unique()}
).data

# Merge gray DOB bars without duplicate legends
for trace in dob_traces:
    trace.legendgroup = trace.name
    trace.showlegend = False
    fig.add_trace(trace)

In [19]:
# === STEP 7: Add Treatment & Beam Annotations (left-aligned) ===
annotations = []
for index, row in df.iterrows():
    annotations.append(dict(
        x=row["Irradiation Date"] - pd.Timedelta(days=230), y=row["Group"],
        text="■■■", showarrow=False,
        font=dict(color=f"rgba(128, 128, 128, {row['Opacity']})", size=18)  # Larger rectangles
    ))
    if row["Treatment"] == treatment_positive_term:
      annotations.append(dict(
          x=row["Irradiation Date"] - pd.Timedelta(days=200), y=row["Group"],
          text=" + ", showarrow=False,
          font=dict(color=row["Treatment_Color"], size=18)  # Larger rectangles
      ))

fig.update_traces(textposition="outside", textfont=dict(size=10))

# === STEP 8: Update Layout & Axis Settings ===
fig.update_layout(
    annotations=annotations,
    width=2140,
    height=800,
    xaxis=dict(
        range=[
            df["Irradiation Date"].min() - pd.Timedelta(days=320),
            df["Termination Date"].max() + pd.Timedelta(days=100)
        ]
    ),
    legend_title_text="Gender | Beam Type | Treatment | Group",
    xaxis_title="Timeline",
    yaxis_title="Group (Ordered)",
    showlegend=True
)

# === STEP 9: Display Chart ===
fig.show()

In [17]:
# === STEP 10: Display Encoding Matrix ===
encoding_matrix = pd.DataFrame({
    "Factor": ["Gender", "Beam Type", "Treatment", "Group", "Subjects", "Dose"],
    "Visual Interpretation": [
        "Base colors: Blue (Male) and Coral Pink (Female)",
        "Opacity: Controls (20% opacity), Silicon Beam (100% opacity)",
        "Treatment Indicator: Sage Green (Treated, NR+), Burnt Orange (Untreated, NR-)",
        "Displayed along Y-axis based on group categorization",
        "Each bar labeled with the number of subjects",
        "Each bar labeled with the respective dose (e.g., '1.5 cGy')"
    ],
    "Example": [
        "Male = #4C72B0, Female = #E9967A",
        "Control Opacity = 0.2, Beam Opacity = 1.0",
        "NR+ = #0D98BA, NR- = #F7BAA8",
        "Groups sorted by 'Order' column",
        "E.g., '10 subjects'",
        "E.g., '2.0 cGy'"
    ]
})

encoding_matrix_fig = go.Figure(data=[go.Table(
    columnwidth=[20, 80, 80],
    header=dict(values=["Factor", "Visual Interpretation", "Example"], fill_color='lightblue', align='left'),
    cells=dict(values=[encoding_matrix[col] for col in encoding_matrix.columns],
               fill_color='lavender', align='left')
)])

encoding_matrix_fig.update_layout(title_text="Encoding Matrix for Graphical Abstract")
encoding_matrix_fig.show()

In [20]:
# === STEP 11: Put it All Together ===
annotations = []
for index, row in df.iterrows():
    annotations.append(dict(
        x=row["Irradiation Date"] - pd.Timedelta(days=230), y=row["Group"],
        text="■■■", showarrow=False,
        font=dict(color=f"rgba(128, 128, 128, {row['Opacity']})", size=18)  # Larger rectangles
    ))
    if row["Treatment"] == treatment_positive_term:
      annotations.append(dict(
          x=row["Irradiation Date"] - pd.Timedelta(days=200), y=row["Group"],
          text=" + ", showarrow=False,
          font=dict(color=row["Treatment_Color"], size=18)  # Larger rectangles
      ))

fig.update_traces(textposition="outside", textfont=dict(size=10))

fig.update_layout(
    annotations=annotations,
    width=2140,
    height=800,
    xaxis=dict(
        range=[
            df["Irradiation Date"].min() - pd.Timedelta(days=320),
            df["Termination Date"].max() + pd.Timedelta(days=100)
        ]
    ),
    legend_title_text="Gender | Beam Type | Treatment | Group",
    xaxis_title="Timeline",
    yaxis_title="Group (Ordered)",
    showlegend=True
)

fig.show()

encoding_matrix = pd.DataFrame({
    "Factor": ["Gender", "Beam Type", "Treatment", "Group", "Subjects", "Dose"],
    "Visual Interpretation": [
        "Base colors: Blue (Male) and Coral Pink (Female)",
        "Opacity: Controls (20% opacity), Silicon Beam (100% opacity)",
        "Treatment Indicator: Sage Green (Treated, NR+), Burnt Orange (Untreated, NR-)",
        "Displayed along Y-axis based on group categorization",
        "Each bar labeled with the number of subjects",
        "Each bar labeled with the respective dose (e.g., '1.5 cGy')"
    ],
    "Example": [
        "Male = #4C72B0, Female = #E9967A",
        "Control Opacity = 0.2, Beam Opacity = 1.0",
        "NR+ = #0D98BA, NR- = #F7BAA8",
        "Groups sorted by 'Order' column",
        "E.g., '10 subjects'",
        "E.g., '2.0 cGy'"
    ]
})

encoding_matrix_fig = go.Figure(data=[go.Table(
    columnwidth=[20, 80, 80],
    header=dict(values=["Factor", "Visual Interpretation", "Example"], fill_color='lightblue', align='left'),
    cells=dict(values=[encoding_matrix[col] for col in encoding_matrix.columns],
               fill_color='lavender', align='left')
)])

encoding_matrix_fig.update_layout(title_text="Encoding Matrix for Graphical Abstract")
encoding_matrix_fig.show()